# Get city statistics from climate data downloaded from CDS
Workflow to get day- and nighttime utci statistics

In [5]:
import sqlite3
import pandas as pd
import xarray as xr
from src import utils
import time
input_folder = "../../../data/"


## Download hourly data

In [6]:
year = '1992'


In [7]:

import cdsapi
c = cdsapi.Client()
args = {
    "months": ['01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',],
    "days":   ['01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31'],
    }
print(year)
c.retrieve(
        'derived-utci-historical', 
    {
        'version': '1_1',
        'format': 'zip',
        'day': args["days"],
        'month': args["months"],
        'year': year,
        'product_type': 'consolidated_dataset',
        'variable': 'universal_thermal_climate_index',
    },
    input_folder+f'utci_hourly_{year}.zip')


1992


2023-07-27 17:07:12,447 INFO Welcome to the CDS
2023-07-27 17:07:12,448 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/derived-utci-historical
2023-07-27 17:07:12,801 INFO Request is queued
2023-07-27 17:07:13,853 INFO Request is running
2023-07-27 17:11:32,389 INFO Request is completed
2023-07-27 17:11:32,391 INFO Downloading https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data2/dataset-derived-utci-historical-09cfa0f8-b96c-4cd6-a9d3-10aa27924540.zip to ../../../data/utci_hourly_1992.zip (14.4G)
2023-07-27 17:38:18,208 INFO Download rate 9.2M/s     


Result(content_length=15416926517,content_type=application/zip,location=https://download-0011-clone.copernicus-climate.eu/cache-compute-0011/cache/data2/dataset-derived-utci-historical-09cfa0f8-b96c-4cd6-a9d3-10aa27924540.zip)

In [8]:

output_folder = utils.unzip_to_folder(input_folder, f'utci_hourly_{year}')

File Name                                             Modified             Size
ECMWF_utci_19920401_v1.1_con.nc                2020-06-26 09:28:22     42276982
ECMWF_utci_19921104_v1.1_con.nc                2020-06-26 09:31:30     42362898
ECMWF_utci_19921203_v1.1_con.nc                2020-06-26 09:33:26     42442964
ECMWF_utci_19920609_v1.1_con.nc                2020-06-26 09:25:42     41875415
ECMWF_utci_19920718_v1.1_con.nc                2020-06-26 09:27:36     41833324
ECMWF_utci_19921016_v1.1_con.nc                2020-06-26 09:28:22     42167837
ECMWF_utci_19920422_v1.1_con.nc                2020-06-26 09:21:00     42206356
ECMWF_utci_19920727_v1.1_con.nc                2020-06-26 09:24:54     41684724
ECMWF_utci_19920911_v1.1_con.nc                2020-06-26 09:29:20     41993965
ECMWF_utci_19920617_v1.1_con.nc                2020-06-26 09:23:12     41741528
ECMWF_utci_19920415_v1.1_con.nc                2020-06-26 09:23:42     42267928
ECMWF_utci_19920824_v1.1_con.nc         

## Get city coordinates

In [9]:
%time
# path to databases
city_geom  = input_folder+'/city_pts_urban_audit2021.sqlite'
con = sqlite3.connect(city_geom)
# read full table
city_all = pd.read_sql_query("SELECT _wgs84x, _wgs84y, city_code FROM urau_lb_2021_3035_cities_center_points_4", con)
con.close()
# get city coordinates
# lonlat_list =[["NL005C", 4.640960, 52.113299], ["NL006C", 5.384670, 52.173656], ["NL007C", 5.921886, 52.189884]]
lon_list = city_all["_wgs84x"].values.tolist()
lat_list = city_all["_wgs84y"].values.tolist()
city_list = city_all["city_code"].values.tolist()
target_lon = xr.DataArray(lon_list, dims="city", coords={"city": city_list})
target_lat = xr.DataArray(lat_list, dims="city", coords={"city": city_list})

CPU times: total: 0 ns
Wall time: 0 ns


## Read the downloaded .nc file with xarray

In [11]:
output_folder = "C:/Users/MariaRicci/Projects_Cdrive/FAIRiCube/data/"
climate_path = output_folder+"utci_hourly_1992/ECMWF_utci*_v1.1_con.nc"
data = xr.open_mfdataset(climate_path, engine="netcdf4", parallel=True)

In [12]:
data

<xarray.Dataset>
Dimensions:  (time: 8784, lon: 1440, lat: 601)
Coordinates:
  * time     (time) datetime64[ns] 1992-01-01 ... 1992-12-31T23:00:00
  * lon      (lon) float64 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -59.25 -59.5 -59.75 -60.0
Data variables:
    utci     (time, lat, lon) float32 dask.array<chunksize=(24, 601, 1440), meta=np.ndarray>
Attributes:
    CDI:                       Climate Data Interface version 1.9.5 (http://m...
    Conventions:               CF-1.6
    institution:               European Centre for Medium-Range Weather Forec...
    CDO:                       Climate Data Operators version 1.9.5 (http://m...
    cdo_openmp_thread_number:  8
    history:                   Fri Jun 26 09:35:02 2020: ncatted -a _FillValu...
    NCO:                       netCDF Operators version 4.7.8 (Homepage = htt...

## Compute statistics

In [13]:
data_cities = data["utci"].sel(lon=target_lon, lat=target_lat, method="ffill")
data_cities_daytime = data_cities.resample(time="12H", base = 7)
utci_mean = data_cities_daytime.mean()
utci_min = data_cities_daytime.min()
utci_max = data_cities_daytime.max()


In [14]:
data_cities

<xarray.DataArray 'utci' (time: 8784, city: 729)>
dask.array<transpose, shape=(8784, 729), dtype=float32, chunksize=(24, 729), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-01-01 ... 1992-12-31T23:00:00
    lon      (city) float64 4.5 5.25 5.75 5.75 6.5 ... 14.75 12.5 9.25 6.5 5.0
    lat      (city) float64 52.25 52.25 52.25 52.0 53.25 ... 46.0 46.0 52.5 52.5
  * city     (city) <U6 'NL005C' 'NL006C' 'NL007C' ... 'NL003C' 'NL004C'
Attributes:
    code:     167
    table:    128

In [15]:

stats = xr.merge([utci_mean.rename("utci_mean"), utci_min.rename("utci_min"), utci_max.rename("utci_max")])
stats

<xarray.Dataset>
Dimensions:    (city: 729, time: 733)
Coordinates:
    lon        (city) float64 4.5 5.25 5.75 5.75 6.5 ... 14.75 12.5 9.25 6.5 5.0
    lat        (city) float64 52.25 52.25 52.25 52.0 ... 46.0 46.0 52.5 52.5
  * city       (city) <U6 'NL005C' 'NL006C' 'NL007C' ... 'NL003C' 'NL004C'
  * time       (time) datetime64[ns] 1991-12-31T19:00:00 ... 1992-12-31T19:00:00
Data variables:
    utci_mean  (time, city) float32 dask.array<chunksize=(1, 729), meta=np.ndarray>
    utci_min   (time, city) float32 dask.array<chunksize=(1, 729), meta=np.ndarray>
    utci_max   (time, city) float32 dask.array<chunksize=(1, 729), meta=np.ndarray>
Attributes:
    code:     167
    table:    128

In [16]:
stats

<xarray.Dataset>
Dimensions:    (city: 729, time: 733)
Coordinates:
    lon        (city) float64 4.5 5.25 5.75 5.75 6.5 ... 14.75 12.5 9.25 6.5 5.0
    lat        (city) float64 52.25 52.25 52.25 52.0 ... 46.0 46.0 52.5 52.5
  * city       (city) <U6 'NL005C' 'NL006C' 'NL007C' ... 'NL003C' 'NL004C'
  * time       (time) datetime64[ns] 1991-12-31T19:00:00 ... 1992-12-31T19:00:00
Data variables:
    utci_mean  (time, city) float32 dask.array<chunksize=(1, 729), meta=np.ndarray>
    utci_min   (time, city) float32 dask.array<chunksize=(1, 729), meta=np.ndarray>
    utci_max   (time, city) float32 dask.array<chunksize=(1, 729), meta=np.ndarray>
Attributes:
    code:     167
    table:    128

In [17]:
from dask.distributed import Client
client = Client()  # Connect to distributed cluster and override default

2023-07-28 08:02:18,223 INFO To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2023-07-28 08:02:18,317 INFO State start
2023-07-28 08:02:18,347 INFO   Scheduler at:     tcp://127.0.0.1:62401
2023-07-28 08:02:18,348 INFO   dashboard at:            127.0.0.1:8787
2023-07-28 08:02:18,449 INFO         Start Nanny at: 'tcp://127.0.0.1:62405'
2023-07-28 08:02:18,456 INFO         Start Nanny at: 'tcp://127.0.0.1:62406'
2023-07-28 08:02:18,459 INFO         Start Nanny at: 'tcp://127.0.0.1:62404'
2023-07-28 08:02:18,462 INFO         Start Nanny at: 'tcp://127.0.0.1:62407'
2023-07-28 08:02:37,614 INFO Register worker <WorkerState 'tcp://127.0.0.1:62427', name: 3, status: init, memory: 0, processing: 0>
2023-07-28 08:02:37,621 INFO Starting worker compute stream, tcp://127.0.0.1:62427
2023-07-28 08:02:37,622 INFO Starting established connection to tcp://127.0.0.1:62432
2023-07-28 08:02:37,625 INFO Register worker <WorkerStat

In [18]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 23.94 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:62401,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 23.94 GiB
Comm: tcp://127.0.0.1:62426,Total threads: 2
Dashboard: http://127.0.0.1:62430/status,Memory: 5.99 GiB
Nanny: tcp://127.0.0.1:62404,


In [19]:
stats_df = stats.to_dataframe()
stats_df

2023-07-28 08:07:14,827 INFO Event loop was unresponsive in Scheduler for 4.44s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2023-07-28 08:07:14,829 INFO Event loop was unresponsive in Nanny for 4.44s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2023-07-28 08:07:14,830 INFO Event loop was unresponsive in Nanny for 4.44s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2023-07-28 08:07:14,831 INFO Event loop was unresponsive in Nanny for 4.44s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
2023-07-28 08:07:14,832 INFO Event loop was unresponsive in Nanny for 4.44s.  This is often caused by long-running GIL-holding functions or moving large chu

lon    lat   utci_mean    utci_min    utci_max
city   time                                                               
NL005C 1991-12-31 19:00:00  4.5  52.25  260.830750  258.663971  262.960938
       1992-01-01 07:00:00  4.5  52.25  262.342682  258.436615  266.392426
       1992-01-01 19:00:00  4.5  52.25  261.008209  259.498749  262.231140
       1992-01-02 07:00:00  4.5  52.25  263.137421  261.100433  266.184937
       1992-01-02 19:00:00  4.5  52.25  258.725220  256.209717  261.414185
...                         ...    ...         ...         ...         ...
NL004C 1992-12-29 19:00:00  5.0  52.50  260.779266  259.389832  261.772888
       1992-12-30 07:00:00  5.0  52.50  267.024323  260.125183  274.717346
       1992-12-30 19:00:00  5.0  52.50  261.214172  259.204529  261.857452
       1992-12-31 07:00:00  5.0  52.50  269.872040  257.206787  282.333679
       1992-12-31 19:00:00  5.0  52.50  267.354309  266.870483  267.767944

[534357 rows x 5 columns]

In [20]:
stats_df = stats_df.reset_index()
stats_df

,city,time,lon,lat,utci_mean,utci_min,utci_max
0,NL005C,1991-12-31 19:00:00,4.5,52.25,260.830750,258.663971,262.960938
1,NL005C,1992-01-01 07:00:00,4.5,52.25,262.342682,258.436615,266.392426
2,NL005C,1992-01-01 19:00:00,4.5,52.25,261.008209,259.498749,262.231140
3,NL005C,1992-01-02 07:00:00,4.5,52.25,263.137421,261.100433,266.184937
4,NL005C,1992-01-02 19:00:00,4.5,52.25,258.725220,256.209717,261.414185
...,...,...,...,...,...,...,...
534352,NL004C,1992-12-29 19:00:00,5.0,52.50,260.779266,259.389832,261.772888
534353,NL004C,1992-12-30 07:00:00,5.0,52.50,267.024323,260.125183,274.717346
534354,NL004C,1992-12-30 19:00:00,5.0,52.50,261.214172,259.204529,261.857452
534355,NL004C,1992-12-31 07:00:00,5.0,52.50,269.872040,257.206787,282.333679


In [21]:
import geopandas as gpd
gdf = gpd.GeoDataFrame(
    stats_df[["city", "time", "utci_mean", "utci_min", "utci_max"]], geometry=gpd.points_from_xy(stats_df.lon,stats_df.lat), crs="EPSG:4326")

In [22]:
gdf.to_file(f"stats_{year}.shp", driver="GeoJSON")

2023-07-28 08:11:29,937 - tornado.application - ERROR - Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.BokehTornado object at 0x0000021FD15D2790>>
Traceback (most recent call last):
  File "c:\Users\MariaRicci\anaconda3\envs\fairicube_env\lib\site-packages\tornado\ioloop.py", line 921, in _run
    val = self.callback()
  File "c:\Users\MariaRicci\anaconda3\envs\fairicube_env\lib\site-packages\bokeh\server\tornado.py", line 760, in _keep_alive
    c.send_ping()
  File "c:\Users\MariaRicci\anaconda3\envs\fairicube_env\lib\site-packages\bokeh\server\connection.py", line 93, in send_ping
    self._socket.ping(str(self._ping_count).encode("utf-8"))
  File "c:\Users\MariaRicci\anaconda3\envs\fairicube_env\lib\site-packages\tornado\websocket.py", line 444, in ping
    raise WebSocketClosedError()
tornado.websocket.WebSocketClosedError
2023-07-28 08:11:29,937 ERROR Exception in callback <bound method BokehTornado._keep_alive of <bokeh.server.tornado.Bokeh

In [23]:
collections = {
        "c_city_utci":
        {
            "crs": 4326,
            "properties":
            {
                "id": "integer",
                "timestamp": "timestamp",
                "utci_mean": "float",
                "utci_min": "float",
                "utci_max": "float",
                "urau_code": "text",
            }
        }
    }

In [27]:
import shutil
del data
del data_cities
del data_cities_daytime
client.close()


NameError: name 'data' is not defined

In [28]:
del data_cities
del data_cities_daytime

In [29]:

shutil.rmtree(input_folder+f"utci_hourly_{year}", ignore_errors=False, onerror=None)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: '../../../data/utci_hourly_1992\\ECMWF_utci_19920113_v1.1_con.nc'